This notebook generates and runs queries to download Gaia data

In [ ]:
import os
from os import path
import glob

# Third-party
import astropy.coordinates as coord
from astropy.table import Table, vstack
from astropy.io import fits
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import gala.coordinates as gc
import gala.dynamics as gd
from pyia import GaiaData

from astroquery.gaia import Gaia

login_file = path.expanduser('~/.gaia/archive.login')
if path.exists(login_file):
    with open(login_file) as f:
        username = f.readline()
        password = f.readline()
    print('Logging in with user: {0}'.format(username))
    Gaia.login(user=username, password=password)

In [ ]:
data_path = '../data/gd1-polygon/'
os.makedirs(data_path, exist_ok=True)

In [ ]:
keep_cols = ['source_id', 
             'ra', 'dec', 'parallax', 'parallax_error', 
             'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_parallax_corr', 'ra_pmra_corr', 
             'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 
             'parallax_pmdec_corr', 'pmra_pmdec_corr', 'visibility_periods_used', 
             'phot_g_mean_mag', 'phot_g_mean_flux_over_error', 
             'phot_bp_mean_mag', 'phot_bp_mean_flux_over_error', 
             'phot_rp_mean_mag', 'phot_rp_mean_flux_over_error', 
             'phot_bp_rp_excess_factor']

In [ ]:
q_base ='''SELECT {1} FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > -0.5 AND bp_rp < 2 AND
      CONTAINS(POINT('ICRS', ra, dec), 
               POLYGON('ICRS', 
                       {0[0].ra.degree}, {0[0].dec.degree}, 
                       {0[1].ra.degree}, {0[1].dec.degree}, 
                       {0[2].ra.degree}, {0[2].dec.degree}, 
                       {0[3].ra.degree}, {0[3].dec.degree})) = 1 AND
      (SQRT(POWER(pmra, 2) + POWER(pmdec, 2)) > 2) AND (pmra < 1) AND (pmdec < 1.5)
'''

In [ ]:
queries = []
for l in np.arange(-100, 30, 10):
    corners = gc.GD1(phi1=[l, l, l+10, l+10]*u.deg, 
                     phi2=[-10, 5, 5, -10]*u.deg)
    corners_icrs = corners.transform_to(coord.ICRS)
    q = q_base.format(corners_icrs, ', '.join(keep_cols))
    print(l)

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        job = Gaia.launch_job_async(q, name='GD1-{0}')

    tbl = job.get_results()

    for c in tbl.colnames: # hack to make sure object arrays are string
        if tbl[c].dtype == object:
            tbl[c] = np.array(tbl[c]).astype(str)
    tbl.write(path.join(data_path, 'gd1-{0:.0f}.fits'.format(l)), overwrite=True)

# Combine tables

In [ ]:
from numpy.lib.recfunctions import stack_arrays

In [ ]:
all_filename = '../data/gd1-polygon/gd1-all.fits'
if not path.exists(all_filename):
    arrs = []
    for filename in glob.glob('../data/gd1-polygon/*.fits'):
        arrs.append(np.array(fits.getdata(filename))[keep_cols])
    arr = stack_arrays(arrs, asrecarray=True, usemask=False)
    t = Table(arr)
    t.write(all_filename)

In [ ]:
t = Table.read(all_filename)

In [ ]:
c_gd1 = coord.SkyCoord(ra=t['ra']*u.deg, dec=t['dec']*u.deg).transform_to(gc.GD1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))

ax.plot(c_gd1.phi1.wrap_at(180*u.deg)[t['phot_g_mean_mag'] < 18],
        c_gd1.phi2[t['phot_g_mean_mag'] < 18], 
        marker=',', linestyle='none', alpha=0.04)

ax.set_xlim(-120, 50)
ax.set_ylim(-10, 5)

ax.set_aspect('equal')